In [1]:
from web3 import Web3
import sys
import json
from web3.middleware import geth_poa_middleware
from config import *

In [2]:
'''
Connects
'''
# To provider

ropstenW3 = Web3(Web3.HTTPProvider(ropstenProvider))
quorumW3 = Web3(Web3.HTTPProvider(quorumProvider))
quorumW3.middleware_onion.inject(geth_poa_middleware, layer=0) # POA chain need additional middleware

# To contracts

f = open('./abi/ERC20_ABI.json')
abi = json.load(f)
quorum20Con = quorumW3.eth.contract(address=quorum20Addr, abi=abi)
f.close()

f = open('./abi/HOME_AMB_E2E_ABI.json')
target = json.load(f)
abi = target['contracts']['/contracts/homeAmbE2E_f.sol:HomeAMBErc677ToErc677']['abi']
ropstenMedCon = ropstenW3.eth.contract(address=ropstenMedAddr, abi=abi)
quorumMedCon = quorumW3.eth.contract(address=quorumMedAddr, abi=abi)
f.close()

f = open('./abi/PermittableToken_ABI.json')
target = json.load(f)
abi = target['contracts']['/contracts/PermittableToken_f.sol:PermittableToken']['abi']
ropsten677Con = ropstenW3.eth.contract(address=ropsten677Addr, abi=abi)
f.close()

#f = open('./abi/677ABI.json')
#target = json.load(f)
#abi = target['contracts']['/contracts/677_f.sol:ERC677']['abi']
# only used for filtering event Transfer
# home677Con's event Transfer is overloaded, which is not supported by contract.event.<myEvent> object
#I677Con = homeW3.eth.contract(address=home677Addr, abi=abi) 
#f.close()

f = open('./abi/HomeAMB_ABI.json')
target = json.load(f)
abi = target['contracts']['/contracts/HomeAMB_f.sol:HomeAMB']['abi']
ropstenBriCon = ropstenW3.eth.contract(address=ropstenBriAddr, abi=abi)
f.close()

f = open('./abi/ForeignAMB_ABI.json')
target = json.load(f)
abi = target['contracts']['/contracts/ForeignAMB_f.sol:ForeignAMBWithGasToken']['abi']
quorumBriCon = quorumW3.eth.contract(address=quorumBriAddr, abi=abi)
f.close()

f = open('./abi/Validator_ABI.json')
target = json.load(f)
abi = target['contracts']['/contracts/BridgeValidators_f.sol:BridgeValidators']['abi']
quorumValCon = quorumW3.eth.contract(address=quorumValAddr, abi=abi)
ropstenValCon = ropstenW3.eth.contract(address=ropstenValAddr, abi=abi)
f.close()

In [3]:
denominator = 10**18
print('ropsten addr1:   ', ropsten677Con.functions.balanceOf(addr).call()/denominator)
print('ropsten addr2:   ', ropsten677Con.functions.balanceOf(addr2).call()/denominator)
print('quorum addr1:    ', quorum20Con.functions.balanceOf(addr).call()/denominator)
print('quorum addr2:    ', quorum20Con.functions.balanceOf(addr2).call()/denominator)
print('quorum mediator: ', quorum20Con.functions.balanceOf(quorumMedAddr).call()/denominator)

ropsten addr1:    27.817708509433984
ropsten addr2:    11.963078898662921
quorum addr1:     1.0
quorum addr2:     3.0
quorum mediator:  92.0


In [27]:
tx = quorum20Con.functions.approve(
quorumMedAddr,
quorumW3.toWei(5, 'ether')
).buildTransaction(dict({
    'nonce':quorumW3.eth.get_transaction_count(addr),
    'chainId':quorumNonce,
    'from':addr,
})
)
signed_txn = quorumW3.eth.account.sign_transaction(tx, privKey)
tx_hash =  quorumW3.eth.send_raw_transaction(signed_txn.rawTransaction)
tx_receipt = quorumW3.eth.wait_for_transaction_receipt(tx_hash)
#print(tx_receipt)
if(not tx_receipt['status']):
    print(tx_receipt)

In [28]:
tx = quorumMedCon.functions.relayTokens(
addr,
quorumW3.toWei(5, 'ether')
).buildTransaction(dict({
    'nonce':quorumW3.eth.get_transaction_count(addr),
    'chainId':quorumNonce,
    'from':addr,
})
)
signed_txn = quorumW3.eth.account.sign_transaction(tx, privKey)
tx_hash =  quorumW3.eth.send_raw_transaction(signed_txn.rawTransaction)
tx_receipt = quorumW3.eth.wait_for_transaction_receipt(tx_hash)
#print(tx_receipt)
if(not tx_receipt['status']):
    print(tx_receipt)

In [30]:
tx = quorum20Con.functions.transfer(
addr2,
quorumW3.toWei(1, 'ether')
).buildTransaction(dict({
    'nonce':quorumW3.eth.get_transaction_count(addr),
    'chainId':quorumNonce,
    'from':addr,
})
)
signed_txn = quorumW3.eth.account.sign_transaction(tx, privKey)
tx_hash =  quorumW3.eth.send_raw_transaction(signed_txn.rawTransaction)
tx_receipt = quorumW3.eth.wait_for_transaction_receipt(tx_hash)
#print(tx_receipt)
if(not tx_receipt['status']):
    print(tx_receipt)

In [46]:
denominator = 10**18
print('ropsten addr1:   ', ropsten677Con.functions.balanceOf(addr).call()/denominator)
print('ropsten addr2:   ', ropsten677Con.functions.balanceOf(addr2).call()/denominator)
print('quorum addr1:    ', quorum20Con.functions.balanceOf(addr).call()/denominator)
print('quorum addr2:    ', quorum20Con.functions.balanceOf(addr2).call()/denominator)
print('quorum mediator: ', quorum20Con.functions.balanceOf(quorumMedAddr).call()/denominator)

ropsten addr1:    91.0
ropsten addr2:    0.0
quorum addr1:     3.0
quorum addr2:     0.0
quorum mediator:  91.0


In [ ]:
tx = ropsten677Con.functions.transferAndCall(
ropstenMedAddr,
ropstenW3.toWei(1, 'ether'),
"0x"
).buildTransaction(dict({
    'nonce':ropstenW3.eth.get_transaction_count(addr)+10,
    'chainId':ropstenNonce,
    'from':addr
})
)
signed_txn = ropstenW3.eth.account.sign_transaction(tx, privKey)
tx_hash =  ropstenW3.eth.send_raw_transaction(signed_txn.rawTransaction)
tx_receipt = ropstenW3.eth.wait_for_transaction_receipt(tx_hash)
print(tx_hash)
if(not tx_receipt['status']):
    print(tx_receipt)

In [45]:
denominator = 10**18
print('ropsten addr1:   ', ropsten677Con.functions.balanceOf(addr).call()/denominator)
print('ropsten addr2:   ', ropsten677Con.functions.balanceOf(addr2).call()/denominator)
print('quorum addr1:    ', quorum20Con.functions.balanceOf(addr).call()/denominator)
print('quorum addr2:    ', quorum20Con.functions.balanceOf(addr2).call()/denominator)
print('quorum mediator: ', quorum20Con.functions.balanceOf(quorumMedAddr).call()/denominator)

ropsten addr1:    6.0
ropsten addr2:    0.0
quorum addr1:     2.0
quorum addr2:     1.0
quorum mediator:  91.0


In [5]:
ropstenW3.eth.get_transaction_count(addr)


136